# 🏠 부동산 실거래가 예측 대회 - KKH - 외부 데이터 추가
> - 학습 데이터에 존재하는 부동산 관련 결측치 정보를 외부 데이터를 활용해 채워 넣는다.
> - 서울시 공동주택 아파트 정보를 활용한다.
> - [서울시 공동주택 아파트 정보](https://data.seoul.go.kr/dataList/OA-15818/S/1/datasetView.do)
> - kimkihong / helpotcreator@gmail.com / Upstage AI Lab 3기
> - 2024.07.16.화 ~ 2024.07.19.금 19:00

## 라이브러리 & 폰트 설정

- 폰트는 .otf 파일을 직접 위치시켜서 임포트 시켰다.
- 본인은 우분투와 윈도우 두 환경에서 동시에 작업 중인데, 이와 같이 폰트를 설정하면, 문제 없다.

In [169]:
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
fe = fm.FontEntry(fname=r'font/NanumGothic.otf', name='NanumBarunGothic')
fm.fontManager.ttflist.insert(0, fe)
plt.rcParams.update({'font.size': 10, 'font.family': 'NanumBarunGothic'})
plt.rc('font', family='NanumBarunGothic')
import seaborn as sns

# utils
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
import warnings;warnings.filterwarnings('ignore')
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display

# Model
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics

import eli5
from eli5.sklearn import PermutationImportance


# 모든 열을 표시하도록 설정
pd.set_option('display.max_columns', None)

In [170]:
train = pd.read_csv('data/train.csv', encoding='utf-8')
add = pd.read_csv('data/yoonjae_add.csv', encoding='EUC-KR') # 서울시 공동주택 아파트 정보
test = pd.read_csv('data/test.csv', encoding='utf-8')
gps = pd.read_csv('data/jaemyung_newXY_for_train.csv', encoding='utf-8')
solution = pd.read_csv('data/solution.csv', encoding='utf-8') # 실거래가

In [171]:
train.sample(2)

,시군구,번지,본번,부번,아파트명,전용면적(㎡),계약년월,계약일,층,건축년도,도로명,해제사유발생일,등기신청일자,거래유형,중개사소재지,"k-단지분류(아파트,주상복합등등)",k-전화번호,k-팩스번호,단지소개기존clob,k-세대타입(분양형태),k-관리방식,k-복도유형,k-난방방식,k-전체동수,k-전체세대수,k-건설사(시공사),k-시행사,k-사용검사일-사용승인일,k-연면적,k-주거전용면적,k-관리비부과면적,k-전용면적별세대현황(60㎡이하),k-전용면적별세대현황(60㎡~85㎡이하),k-85㎡~135㎡이하,k-135㎡초과,k-홈페이지,k-등록일자,k-수정일자,고용보험관리번호,경비비관리형태,세대전기계약방법,청소비관리형태,건축면적,주차대수,기타/의무/임대/임의=1/2/3/4,단지승인일,사용허가여부,관리비 업로드,좌표X,좌표Y,단지신청일,target
881123,서울특별시 마포구 염리동,521,521.0,0.0,마포자이,84.692,201302,20,15,2003,독막로42길 2,NaN,,-,-,아파트,0232753925,232753926.0,1212.0,분양,위탁관리,계단식,지역난방,10.0,534.0,GS건설(주),진주아파트 재건축조합,2003-12-05 00:00:00.0,105206.0,58250.0,71297.0,0.0,228.0,166.0,NaN,NaN,NaN,2023-09-26 10:31:37.0,NaN,위탁,종합계약,위탁,4145.0,892.0,의무,2013-05-30 18:26:47.0,Y,N,126.945191,37.543329,2013-03-07 09:46:49.0,65000
971206,서울특별시 강남구 청담동,28-13,28.0,13.0,골든아트빌,153.610,201002,9,3,2000,삼성로135길 28,NaN,,-,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,73500



## '시군구' 피처를 이용하여 '구'와 '동' 피처로 나누고, 추가한다.

- '시'는 서울특별시 데이터만 존재하기에 피처 추가하지 않는다.
- 본 과정은 EDA 부분에서 진행할 예정이었으나, 본 과정(외부 데이터 연동)에서 '구' 피처가 필요하여 미리 진행한다.

In [172]:
train['구'] = train['시군구'].apply(lambda x: x.split()[1])
train['동'] = train['시군구'].apply(lambda x: x.split()[2])
display(train.head(2))

,시군구,번지,본번,부번,아파트명,전용면적(㎡),계약년월,계약일,층,건축년도,도로명,해제사유발생일,등기신청일자,거래유형,중개사소재지,"k-단지분류(아파트,주상복합등등)",k-전화번호,k-팩스번호,단지소개기존clob,k-세대타입(분양형태),k-관리방식,k-복도유형,k-난방방식,k-전체동수,k-전체세대수,k-건설사(시공사),k-시행사,k-사용검사일-사용승인일,k-연면적,k-주거전용면적,k-관리비부과면적,k-전용면적별세대현황(60㎡이하),k-전용면적별세대현황(60㎡~85㎡이하),k-85㎡~135㎡이하,k-135㎡초과,k-홈페이지,k-등록일자,k-수정일자,고용보험관리번호,경비비관리형태,세대전기계약방법,청소비관리형태,건축면적,주차대수,기타/의무/임대/임의=1/2/3/4,단지승인일,사용허가여부,관리비 업로드,좌표X,좌표Y,단지신청일,target,구,동
0,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,79.97,201712,8,3,1987,언주로 3,NaN,,-,-,아파트,025776611,025776673,NaN,분양,자치관리,계단식,개별난방,8.0,270.0,우성건설,모름,1987-11-21 00:00:00.0,22637.0,20204.0,22637.0,20.0,250.0,0.0,NaN,NaN,2022-11-09 20:10:43.0,2023-09-23 17:21:41.0,NaN,직영,단일계약,직영,4858.0,262.0,임의,2022-11-17 13:00:29.0,Y,N,127.05721,37.476763,2022-11-17 10:19:06.0,124000,강남구,개포동
1,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,79.97,201712,22,4,1987,언주로 3,NaN,,-,-,아파트,025776611,025776673,NaN,분양,자치관리,계단식,개별난방,8.0,270.0,우성건설,모름,1987-11-21 00:00:00.0,22637.0,20204.0,22637.0,20.0,250.0,0.0,NaN,NaN,2022-11-09 20:10:43.0,2023-09-23 17:21:41.0,NaN,직영,단일계약,직영,4858.0,262.0,임의,2022-11-17 13:00:29.0,Y,N,127.05721,37.476763,2022-11-17 10:19:06.0,123500,강남구,개포동


## '구', '동' 피처가 잘 추가되었는지 train 데이터를 확인한다.

In [173]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1118822 entries, 0 to 1118821
Data columns (total 54 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   시군구                     1118822 non-null  object 
 1   번지                      1118597 non-null  object 
 2   본번                      1118747 non-null  float64
 3   부번                      1118747 non-null  float64
 4   아파트명                    1116696 non-null  object 
 5   전용면적(㎡)                 1118822 non-null  float64
 6   계약년월                    1118822 non-null  int64  
 7   계약일                     1118822 non-null  int64  
 8   층                       1118822 non-null  int64  
 9   건축년도                    1118822 non-null  int64  
 10  도로명                     1118822 non-null  object 
 11  해제사유발생일                 5983 non-null     float64
 12  등기신청일자                  1118822 non-null  object 
 13  거래유형                    1118822 non-null  object 
 14  중개

In [174]:
train.columns

Index(['시군구', '번지', '본번', '부번', '아파트명', '전용면적(㎡)', '계약년월', '계약일', '층', '건축년도',
       '도로명', '해제사유발생일', '등기신청일자', '거래유형', '중개사소재지', 'k-단지분류(아파트,주상복합등등)',
       'k-전화번호', 'k-팩스번호', '단지소개기존clob', 'k-세대타입(분양형태)', 'k-관리방식', 'k-복도유형',
       'k-난방방식', 'k-전체동수', 'k-전체세대수', 'k-건설사(시공사)', 'k-시행사', 'k-사용검사일-사용승인일',
       'k-연면적', 'k-주거전용면적', 'k-관리비부과면적', 'k-전용면적별세대현황(60㎡이하)',
       'k-전용면적별세대현황(60㎡~85㎡이하)', 'k-85㎡~135㎡이하', 'k-135㎡초과', 'k-홈페이지',
       'k-등록일자', 'k-수정일자', '고용보험관리번호', '경비비관리형태', '세대전기계약방법', '청소비관리형태',
       '건축면적', '주차대수', '기타/의무/임대/임의=1/2/3/4', '단지승인일', '사용허가여부', '관리비 업로드',
       '좌표X', '좌표Y', '단지신청일', 'target', '구', '동'],
      dtype='object')

In [175]:
print((train.isnull().mean() * 100).to_string())

시군구                        0.000000
번지                         0.020110
본번                         0.006703
부번                         0.006703
아파트명                       0.190021
전용면적(㎡)                    0.000000
계약년월                       0.000000
계약일                        0.000000
층                          0.000000
건축년도                       0.000000
도로명                        0.000000
해제사유발생일                   99.465241
등기신청일자                     0.000000
거래유형                       0.000000
중개사소재지                     0.000000
k-단지분류(아파트,주상복합등등)        77.822120
k-전화번호                    77.784849
k-팩스번호                    78.005438
단지소개기존clob                93.870160
k-세대타입(분양형태)              77.721300
k-관리방식                    77.721300
k-복도유형                    77.750527
k-난방방식                    77.721300
k-전체동수                    77.816668
k-전체세대수                   77.721300
k-건설사(시공사)                77.854922
k-시행사                     77.872441
k-사용검사일-사용승인일             77

In [176]:
train = pd.concat([train, gps], axis=1)
train = train.drop(['좌표X', '좌표Y'], axis=1)
train = train.rename(columns={'좌표X_2': '좌표X'})
train = train.rename(columns={'좌표Y_2': '좌표Y'})

In [177]:
print((train.isnull().mean() * 100).to_string())

시군구                        0.000000
번지                         0.020110
본번                         0.006703
부번                         0.006703
아파트명                       0.190021
전용면적(㎡)                    0.000000
계약년월                       0.000000
계약일                        0.000000
층                          0.000000
건축년도                       0.000000
도로명                        0.000000
해제사유발생일                   99.465241
등기신청일자                     0.000000
거래유형                       0.000000
중개사소재지                     0.000000
k-단지분류(아파트,주상복합등등)        77.822120
k-전화번호                    77.784849
k-팩스번호                    78.005438
단지소개기존clob                93.870160
k-세대타입(분양형태)              77.721300
k-관리방식                    77.721300
k-복도유형                    77.750527
k-난방방식                    77.721300
k-전체동수                    77.816668
k-전체세대수                   77.721300
k-건설사(시공사)                77.854922
k-시행사                     77.872441
k-사용검사일-사용승인일             77

## 도로명 결측치를 해결한다.

- 추가 데이터와 합치기 위해, 도로명 주소에 존재하는 결측치를 미리 해결해두기 위함이다.

## 우선, 도로명 데이터가 결측치는 아니면서, 이상치 데이터들이 존재한다.

- 도로명 데이터 길이가 3 이하인 데이터들을 찾아서, 결측치로 바꾼다.
- 도로명 데이터가 '~로' 형태로 끝나는 데이터들을 찾아서, 결측치로 바꾼다.

In [178]:
train['도로명'].apply(lambda x: len(x) <= 4).sum()

2041

In [179]:
train['도로명'] = train['도로명'].apply(lambda x: np.nan if len(x) <= 4 else x)

## 동일한 아파트인데, 일부 데이터만 도로명 주소가 결측치인 경우를 조회한다.

In [180]:
# '아파트명' 및 '동'으로 그룹화하고 '도로명'의 유니크 값을 리스트에 담기
grouped = train.groupby(['아파트명', '동'])['도로명'].unique().reset_index()

# '도로명'의 유니크 값이 2 이상인 '아파트명'을 찾고 도로명별 데이터 개수 출력
for index, row in grouped.iterrows():
    apartment_name = row['아파트명']
    dong_name = row['동']
    unique_road_names = row['도로명']
    
    # 아파트명에 해당하는 원본 데이터에서 '도로명'별 개수 계산
    road_counts = train[train['아파트명'] == apartment_name]['도로명'].value_counts()
    
    # if len(unique_road_names) > 1:
    if (len(unique_road_names) > 1) and (' ' in unique_road_names):
        print(f"아파트명: {apartment_name} / 동: {dong_name}")
        for road in unique_road_names:
            print(f"- {road_counts.get(road, 0)}개 / 도로명: '{road}'")
        print("-" * 40)

## 동일한 아파트 데이터 중에서, 가장 많이 적혀 있는 도로명 주소를 복사하여 결측치를 채운다.

In [181]:
# ' ' 값을 NaN으로 변환
train['도로명'] = train['도로명'].replace(' ', np.nan)


def fill_road_name():
    # '아파트명' 및 '동'으로 그룹화하고 '도로명'의 유니크 값을 리스트에 담기
    grouped = train.groupby(['아파트명', '동'])['도로명'].unique().reset_index()

    # '도로명'의 유니크 값이 2 이상인 '아파트명'을 찾고 도로명별 데이터 개수 출력
    for index, row in grouped.iterrows():
        apartment_name = row['아파트명']
        dong_name = row['동']
        unique_road_names = row['도로명']
        
        if len(unique_road_names) > 1:
            # 아파트명에 해당하는 원본 데이터에서 '도로명'별 개수 계산
            road_counts = train[train['아파트명'] == apartment_name]['도로명'].value_counts()
            
            if any(pd.isna(unique_road_names)):
                print(f"아파트명: {apartment_name} / 동: {dong_name}")
                for road in unique_road_names:
                    if pd.notna(road):
                        print(f"- {road_counts.get(road, 0)}개 / 도로명: '{road}'")
                print("-" * 40)
                
                # 결측치 채우기
                most_common_road = road_counts.idxmax()
                train.loc[(train['아파트명'] == apartment_name) & (train['동'] == dong_name) & (train['도로명'].isna()), '도로명'] = most_common_road

fill_road_name()

아파트명: DMC롯데캐슬더퍼스트 / 동: 수색동
- 41개 / 도로명: '수색로 300'
----------------------------------------
아파트명: DMC파크뷰자이1단지 / 동: 남가좌동
- 832개 / 도로명: '가재울미래로 2'
----------------------------------------
아파트명: e편한세상송파파크센트럴 / 동: 거여동
- 30개 / 도로명: '오금로 551'
----------------------------------------
아파트명: 강남엘에이치1단지 / 동: 세곡동
- 264개 / 도로명: '헌릉로571길 20'
----------------------------------------
아파트명: 강남한신휴플러스6단지 / 동: 율현동
- 73개 / 도로명: '밤고개로26길 50'
----------------------------------------
아파트명: 강남한신휴플러스8단지 / 동: 율현동
- 8개 / 도로명: '밤고개로27길 20'
----------------------------------------
아파트명: 강남한양수자인 / 동: 자곡동
- 360개 / 도로명: '자곡로 260'
----------------------------------------
아파트명: 개포주공1단지 / 동: 개포동
- 1824개 / 도로명: '개포로 310'
- 448개 / 도로명: '선릉로 7'
----------------------------------------
아파트명: 관악동부센트레빌 / 동: 봉천동
- 487개 / 도로명: '은천로33길 5'
----------------------------------------
아파트명: 관악푸르지오102동 / 동: 사당동
- 18개 / 도로명: '관악로30길 27'
----------------------------------------
아파트명: 꿈의숲SKVIEW / 동: 월계동
- 215개 / 도로명: '월계로42길 97'
-----------

## 그 외에, '도로명'이 결측치인 데이터를 아파트별로 찾아낸다.

In [182]:
display(train[train['도로명'].isna()].groupby(['시군구', '아파트명']).size().reset_index(name='count'))

,시군구,아파트명,count
0,서울특별시 강남구 역삼동,진달래2차,6
1,서울특별시 강남구 역삼동,진달래3차,2
2,서울특별시 관악구 봉천동,관악푸르지오102동,2
3,서울특별시 동대문구 답십리동,태양,1
4,서울특별시 마포구 아현동,현대,34
5,서울특별시 마포구 토정동,마포지구시범6동,1
6,서울특별시 서초구 반포동,래미안원베일리,1
7,서울특별시 서초구 서초동,금호,3
8,서울특별시 성북구 돈암동,문화,15
9,서울특별시 송파구 송파동,반도1차,50


## 이 아파트들은 도로명 주소를 직접 찾아내야 한다.

- 서울특별시 강남구 역삼동 / 진달래2차     --> 없어진 아파트
- 서울특별시 강남구 역삼동 / 진달래3차     --> 없어진 아파트
- 서울특별시 동대문구 답십리동 / 태양      --> 없어진 아파트
- 서울특별시 마포구 아현동 / 현대          --> 없어진 아파트
- 서울특별시 마포구 토정동 / 마포지구시범6동 --> 없어진 아파트
- 서울특별시 서초구 반포동 / 래미안원베일리 --> 존재함!! 도로명 주소: 반포대로 333
- 서울특별시 서초구 서초동 / 금호 --> 없어진 아파트 (서초금호어울림아파트인가? 아니다. 서초금호어울림아파트는 따로 데이터가 존재한다.)
- 서울특별시 성북구 돈암동 / 문화          --> 없어진 아파트
- 서울특별시 송파구 송파동 / 반도1차       --> 없어진 아파트
- 서울특별시 영등포구 당산동3가 / 평화     --> 없어진 아파트 '평화아파트리모델링주택조합'이라는 아파트가 새롭게 생김.
- 서울특별시 용산구 한강로2가 / 신용산빌딩 --> 없어진 아파트
- 서울특별시 종로구 옥인동 / 옥인시민      --> 없어진 아파트

## 래미안원베일리 아파트는 도로명을 찾았고, 채워 넣는다.

In [183]:
mask = (train['시군구'] == '서울특별시 서초구 반포동') & (train['아파트명'].str.contains('래미안원베일리'))
train.loc[mask & train['도로명'].isna(), '도로명'] = '반포대로 333'

## 없어진 아파트 데이터 확인

- 우선, 지금 더 진행할 수 있는 내용은 없다.
- 계약년월을 확인해보니, 아주 오래된 데이터이니, 삭제하는게 좋아 보인다.

In [184]:
display(train[train['도로명'].isna()][['시군구', '아파트명', '도로명', '계약년월']])

,시군구,아파트명,도로명,계약년월
172114,서울특별시 마포구 아현동,현대,NaN,200811
172115,서울특별시 마포구 아현동,현대,NaN,200811
172116,서울특별시 마포구 아현동,현대,NaN,200901
172117,서울특별시 마포구 아현동,현대,NaN,200902
172118,서울특별시 마포구 아현동,현대,NaN,200902
...,...,...,...,...
1115468,서울특별시 종로구 옥인동,옥인시민,NaN,200708
1115469,서울특별시 종로구 옥인동,옥인시민,NaN,200708
1115470,서울특별시 종로구 옥인동,옥인시민,NaN,200709
1115471,서울특별시 종로구 옥인동,옥인시민,NaN,200709


In [185]:
train = train.dropna(subset=['도로명'])
display(train[train['도로명'].isna()][['시군구', '아파트명', '도로명', '계약년월']])

,시군구,아파트명,도로명,계약년월


## 이제 외부 데이터를 확인한다.

In [186]:
add.head(2)

,번호,k-아파트코드,k-아파트명,"k-단지분류(아파트,주상복합등등)",kapt도로명주소,주소(시도)k-apt주소split,주소(시군구),주소(읍면동),나머지주소,주소(도로명),주소(도로상세주소),k-전화번호,k-팩스번호,단지소개기존clob,단지첨부파일,k-세대타입(분양형태),k-관리방식,k-복도유형,k-난방방식,k-전체동수,k-전체세대수,k-건설사(시공사),k-시행사,k-사용검사일-사용승인일,k-연면적,k-주거전용면적,k-관리비부과면적,k-전용면적별세대현황(60㎡이하),k-전용면적별세대현황(60㎡~85㎡이하),k-85㎡~135㎡이하,k-135㎡초과,k-홈페이지,k-등록일자,k-수정일자,고용보험관리번호,경비비관리형태,세대전기계약방법,청소비관리형태,건축면적,주차대수,기타/의무/임대/임의=1/2/3/4,단지승인일,사용허가여부,관리비 업로드,좌표X,좌표Y,단지신청일
0,1,A15679103,우리유앤미,아파트,서울특별시 동작구 서달로 83,서울,동작구,흑석동,우리유앤미아파트,서달로,83,028127541,028127542,NaN,NaN,분양,위탁관리,혼합식,개별난방,2.0,206.0,우리건설,경수재건축조합,2003-12-26 00:00:00.0,27097.0,15827.0,20098.0,89.0,93.0,24.0,NaN,NaN,NaN,2024-07-11 12:44:31.0,90800610361,위탁,단일계약,위탁,1773.56,223.0,의무,2018-04-10 15:59:42.0,Y,N,126.9596386,37.500668,2013-03-07 09:46:59.0
1,2,A13876112,송파파인타운13단지,아파트,서울특별시 송파구 송파대로8길 10,서울,송파구,장지동,857,송파대로8길,10,024002658,024002668,NaN,NaN,분양,위탁관리,계단식,개별난방,4.0,197.0,양우건설(주),SH공사,2011-01-27 00:00:00.0,30646.0,16720.0,22520.0,0.0,197.0,0.0,NaN,NaN,NaN,2024-07-11 19:04:07.0,911-00-18063-1,위탁,단일계약,위탁,0.00,225.0,의무,2013-06-17 19:03:30.0,Y,N,127.1291789,37.476897,2013-03-07 09:46:59.0


In [187]:
add.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2792 entries, 0 to 2791
Data columns (total 47 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   번호                      2792 non-null   int64  
 1   k-아파트코드                 2792 non-null   object 
 2   k-아파트명                  2792 non-null   object 
 3   k-단지분류(아파트,주상복합등등)      2721 non-null   object 
 4   kapt도로명주소               2661 non-null   object 
 5   주소(시도)k-apt주소split      2792 non-null   object 
 6   주소(시군구)                 2792 non-null   object 
 7   주소(읍면동)                 2792 non-null   object 
 8   나머지주소                   2132 non-null   object 
 9   주소(도로명)                 2679 non-null   object 
 10  주소(도로상세주소)              2672 non-null   object 
 11  k-전화번호                  2785 non-null   object 
 12  k-팩스번호                  2716 non-null   object 
 13  단지소개기존clob              566 non-null    float64
 14  단지첨부파일                  184 non-null    

In [188]:
add.columns

Index(['번호', 'k-아파트코드', 'k-아파트명', 'k-단지분류(아파트,주상복합등등)', 'kapt도로명주소',
       '주소(시도)k-apt주소split', '주소(시군구)', '주소(읍면동)', '나머지주소', '주소(도로명)',
       '주소(도로상세주소)', 'k-전화번호', 'k-팩스번호', '단지소개기존clob', '단지첨부파일',
       'k-세대타입(분양형태)', 'k-관리방식', 'k-복도유형', 'k-난방방식', 'k-전체동수', 'k-전체세대수',
       'k-건설사(시공사)', 'k-시행사', 'k-사용검사일-사용승인일', 'k-연면적', 'k-주거전용면적',
       'k-관리비부과면적', 'k-전용면적별세대현황(60㎡이하)', 'k-전용면적별세대현황(60㎡~85㎡이하)',
       'k-85㎡~135㎡이하', 'k-135㎡초과', 'k-홈페이지', 'k-등록일자', 'k-수정일자', '고용보험관리번호',
       '경비비관리형태', '세대전기계약방법', '청소비관리형태', '건축면적', '주차대수', '기타/의무/임대/임의=1/2/3/4',
       '단지승인일', '사용허가여부', '관리비 업로드', '좌표X', '좌표Y', '단지신청일'],
      dtype='object')

## 외부 데이터도 결측치 있는지 확인

- train 데이터와 합칠 때 key로 사용할 도로명 주소에 결측치가 있음을 확인함
- 도로명 주소를 기준으로도 데이터를 합치고, '동' 기준으로도 데이터를 합치는 방향으로 설정함

In [189]:
print(f"외부 데이터의 'kapt도로명주소' 피처의 결측치 수: ", add['kapt도로명주소'].isnull().sum())
print(f"외부 데이터의 '주소(도로명)' 피처의 결측치 수: ",  add['주소(도로명)'].isnull().sum())
print(f"외부 데이터의 '주소(읍면동)' 피처의 결측치 수: ",  add['주소(읍면동)'].isnull().sum())

외부 데이터의 'kapt도로명주소' 피처의 결측치 수:  131
외부 데이터의 '주소(도로명)' 피처의 결측치 수:  113
외부 데이터의 '주소(읍면동)' 피처의 결측치 수:  0


## 데이터를 합쳐줄, 함수 작성

- train 데이터와 add 데이터가 비슷하게 보이지만, 피처명이 다른게 많다.
- 하지만 다행히, 결측치가 많아서 채워넣어야할 뒷 부분은 피처명이 서로 같다.
- k-전화번호 피처를 포함해서 그 이후는 순서도 서로 같고, 피처명도 서로 같다.

In [190]:
# 결측치 확인하고, 채워 넣을 피쳐 리스트 작성
missing_check_list = ['k-전화번호', 'k-팩스번호', '단지소개기존clob', 'k-세대타입(분양형태)', 'k-관리방식', 'k-복도유형',
              'k-난방방식', 'k-전체동수', 'k-전체세대수', 'k-건설사(시공사)', 'k-시행사', 'k-사용검사일-사용승인일',
              'k-연면적', 'k-주거전용면적', 'k-관리비부과면적', 'k-전용면적별세대현황(60㎡이하)',
              'k-전용면적별세대현황(60㎡~85㎡이하)', 'k-85㎡~135㎡이하', 'k-135㎡초과', 'k-홈페이지',
              'k-등록일자', 'k-수정일자', '고용보험관리번호', '경비비관리형태', '세대전기계약방법', '청소비관리형태',
              '건축면적', '주차대수', '기타/의무/임대/임의=1/2/3/4', '단지승인일', '사용허가여부', '관리비 업로드',
              '좌표X', '좌표Y', '단지신청일']

def missing_update(key_feature: str):
    # 결측치 채우기 전 결측치 수 및 비율 확인
    total_rows = len(train)
    missing_before = train[missing_check_list].isnull().sum().reset_index()
    missing_before.columns = ['Feature', 'Missing_Before']
    missing_before['Missing_Before_%'] = (missing_before['Missing_Before'] / total_rows) * 100

    # 결측치 채우기
    for feature in missing_check_list:
        if feature in add.columns:
            add_feature_dict = add.set_index(key_feature)[feature].to_dict()
            train[feature] = train.apply(lambda row: add_feature_dict.get(row[key_feature], row[feature]) if pd.isnull(row[feature]) else row[feature], axis=1)

    # 결측치 채우기 후 결측치 수 및 비율 확인
    missing_after = train[missing_check_list].isnull().sum().reset_index()
    missing_after.columns = ['Feature', 'Missing_After']
    missing_after['Missing_After_%'] = (missing_after['Missing_After'] / total_rows) * 100

    # 결측치 비교 테이블 생성
    missing_comparison = missing_before.merge(missing_after, on='Feature')

    # 결측치 비교 결과를 깔끔하게 출력
    display(missing_comparison)

## '동+아파트명' 기준으로 train + add 진행

- 원래는 '아파트명' 기준으로 진행하려 하였으나, '동'을 활용한 이유는, 같은 이름의 아파트가 존재할 수 있다고 판단하여, '동+아파트명'으로 진행함.
- 같은 아파트의 '층'에 따라서도 거래가격이 다르니, '층'을 함께 넣어야 하나, add 데이터에는 '층'이 없어서, '층'은 넣지 못함.
- 본 과정을 통해, 피처별로 결측치가 소폭 줄어 들었음.

In [191]:
# 새로운 피처 '동+아파트명' 생성
train['동+아파트명'] = train['동'] + ' ' + train['아파트명']
add['동+아파트명'] = add['주소(읍면동)'] + ' ' + add['k-아파트명']

missing_update('동+아파트명')

# key로 사용한 '동+아파트명' 피처는 삭제한다.
train.drop(columns=['동+아파트명'], inplace=True)


,Feature,Missing_Before,Missing_Before_%,Missing_After,Missing_After_%
0,k-전화번호,870085,77.781095,854113,76.353281
1,k-팩스번호,872553,78.001722,856610,76.576500
2,단지소개기존clob,1050051,93.869124,1042943,93.233706
3,k-세대타입(분양형태),869374,77.717536,853402,76.289721
4,k-관리방식,869374,77.717536,853402,76.289721
5,k-복도유형,869701,77.746768,853729,76.318954
6,k-난방방식,869374,77.717536,853402,76.289721
7,k-전체동수,870441,77.812920,854469,76.385106
8,k-전체세대수,869374,77.717536,853200,76.271664
9,k-건설사(시공사),870869,77.851181,855384,76.466902


## '도로명' 기준으로 train + add 진행

- train 데이터의 '도로명' 피처와 동일한 형태로 add에도 피처 생성함.
- 이를 두 데이터 합치는 key 값으로 사용함.
- 결측치가 76% 이상인 피처들이 대부분 22% 정도로 많이 줄어듬.

In [192]:
add['도로명'] = add['주소(도로명)'] + ' ' + add['주소(도로상세주소)']

missing_update('도로명')

,Feature,Missing_Before,Missing_Before_%,Missing_After,Missing_After_%
0,k-전화번호,854113,76.353281,251751,22.505236
1,k-팩스번호,856610,76.576500,260598,23.296112
2,단지소개기존clob,1042943,93.233706,899224,80.385971
3,k-세대타입(분양형태),853402,76.289721,252216,22.546805
4,k-관리방식,853402,76.289721,252216,22.546805
5,k-복도유형,853729,76.318954,252311,22.555297
6,k-난방방식,853402,76.289721,251618,22.493347
7,k-전체동수,854469,76.385106,253537,22.664895
8,k-전체세대수,853200,76.271664,251370,22.471177
9,k-건설사(시공사),855384,76.466902,255740,22.861832


## 중복 데이터 찾기

- 중복 데이터는 없다.

In [193]:
display("중복 데이터: ", train[train.duplicated()])

'중복 데이터: '

,시군구,번지,본번,부번,아파트명,전용면적(㎡),계약년월,계약일,층,건축년도,도로명,해제사유발생일,등기신청일자,거래유형,중개사소재지,"k-단지분류(아파트,주상복합등등)",k-전화번호,k-팩스번호,단지소개기존clob,k-세대타입(분양형태),k-관리방식,k-복도유형,k-난방방식,k-전체동수,k-전체세대수,k-건설사(시공사),k-시행사,k-사용검사일-사용승인일,k-연면적,k-주거전용면적,k-관리비부과면적,k-전용면적별세대현황(60㎡이하),k-전용면적별세대현황(60㎡~85㎡이하),k-85㎡~135㎡이하,k-135㎡초과,k-홈페이지,k-등록일자,k-수정일자,고용보험관리번호,경비비관리형태,세대전기계약방법,청소비관리형태,건축면적,주차대수,기타/의무/임대/임의=1/2/3/4,단지승인일,사용허가여부,관리비 업로드,단지신청일,target,구,동,좌표X,좌표Y
846076,서울특별시 중랑구 신내동,817,817.0,0.0,신내 데시앙포레,114.54,201312,23,7,2013,신내역로 165,NaN,,-,-,NaN,024968341,024968343,435.0,기타,위탁관리,혼합식,개별난방,23.0,1896.0,(주)태영건설,서울특별시 SH공사,2013-12-11 00:00:00.0,267015.0,133452.0,266279.0,1034.0,610.0,252.0,NaN,NaN,NaN,2024-07-10 09:44:48.0,914-04-74795-1,위탁,종합계약,위탁,NaN,2202.0,의무,2015-03-03 10:13:33.0,Y,N,2014-09-02 15:05:38.0,38061,중랑구,신내동,127.110059,37.615259
846144,서울특별시 중랑구 신내동,817,817.0,0.0,신내 데시앙포레,84.65,201312,30,14,2013,신내역로 165,NaN,,-,-,NaN,024968341,024968343,435.0,기타,위탁관리,혼합식,개별난방,23.0,1896.0,(주)태영건설,서울특별시 SH공사,2013-12-11 00:00:00.0,267015.0,133452.0,266279.0,1034.0,610.0,252.0,NaN,NaN,NaN,2024-07-10 09:44:48.0,914-04-74795-1,위탁,종합계약,위탁,NaN,2202.0,의무,2015-03-03 10:13:33.0,Y,N,2014-09-02 15:05:38.0,33444,중랑구,신내동,127.110059,37.615259


In [194]:
# def impute_missing_values(df):
#     # 수치형 피처와 범주형 피처 구분
#     numeric_features = df.select_dtypes(include=[np.number]).columns
#     categorical_features = df.select_dtypes(include=[object]).columns
    
#     # 수치형 피처의 결측치를 평균값으로 대체
#     for feature in numeric_features:
#         mean_value = df[feature].mean()
#         df[feature].fillna(mean_value, inplace=True)
    
#     # 범주형 피처의 결측치를 'Missing'으로 대체
#     for feature in categorical_features:
#         df[feature].fillna('Missing', inplace=True)
    
#     return df

# impute_missing_values(train)

## 새로운 train 엑셀 파일 저장

In [195]:
# 파일 저장 부분이기 때문에, 실제 저장할 경우만 주석 풀고 사용!
train.to_csv('data/kkh_train.csv', index=False)